In [1]:
import optuna, re, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict

from optuna.importance import get_param_importances
from optuna.visualization.matplotlib import plot_contour as plot_contour_mpl
from matplotlib import gridspec

import sys
sys.path.append('..')

from src.plotting import plot_grouped_importances, plot_contour_search

warnings.filterwarnings('ignore')

/Users/guane/Documentos/Doctorate/MSRRFF-Wind-Forecast/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
METRICS_DIR = Path("../results/metrics")
PLOTS_DIR = Path("../results/plots")

In [3]:
files = sorted(METRICS_DIR.glob("*.db"))  
for file in files:
    size_kb = file.stat().st_size / 1024
    print(f"📄 {file.name:<40} {size_kb:>8.2f} KB")

📄 optuna_results_Argone.db                   152.00 KB
📄 optuna_results_Beijing.db                  152.00 KB
📄 optuna_results_Chengdu.db                  152.00 KB
📄 optuna_results_Netherland-0.db             152.00 KB
📄 optuna_results_Netherland-1.db             152.00 KB
📄 optuna_results_Netherland-2.db             152.00 KB
📄 optuna_results_Synthetic.db                152.00 KB


In [4]:
for db_file in files:
    storage_name = f"sqlite:///{db_file.absolute()}"
    available_studies = optuna.study.get_all_study_names(storage=storage_name)

    name = db_file.stem.replace('optuna_results_', '')
    print(f"\n📊 Processing: {name}")

    # Crear carpeta específica para este dataset
    dataset_plots_dir = PLOTS_DIR / name
    dataset_plots_dir.mkdir(parents=True, exist_ok=True)

    all_importances: Dict[str, Dict[str, float]] = {}

    for study_name in available_studies:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        importances = get_param_importances(study)
        all_importances[study_name] = importances

        fig = optuna.visualization.plot_contour(study, params=None)
        fig.update_layout(width=900, height=800)
        # Guardar en la carpeta específica del dataset (PDF soportado por Kaleido v1)
        contour_path = dataset_plots_dir / f"{study_name}_contour.eps"
        fig.write_image(str(contour_path))

    # Generar gráfica agrupada de importancias por base de datos en su carpeta
    plot_grouped_importances(all_importances, str(dataset_plots_dir / "importances.eps"))
    
    print(f"  ✅ Contour + grouped importances saved in {name}/")
    



📊 Processing: Argone
['RFF-RNN Tuning-Argone', 'RFF-GRU Tuning-Argone', 'RFF-LSTM Tuning-Argone']


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


  ✅ Contour + grouped importances saved in Argone/

📊 Processing: Beijing
['RFF-RNN Tuning-Beijing', 'RFF-GRU Tuning-Beijing', 'RFF-LSTM Tuning-Beijing']


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


  ✅ Contour + grouped importances saved in Beijing/

📊 Processing: Chengdu
['RFF-RNN Tuning-Chengdu', 'RFF-GRU Tuning-Chengdu', 'RFF-LSTM Tuning-Chengdu']


...

['RFF-RNN Tuning-Netherland-1', 'RFF-GRU Tuning-Netherland-1', 'RFF-LSTM Tuning-Netherland-1']
  ✅ Contour + grouped importances saved in Netherland-1/

📊 Processing: Netherland-2


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


['RFF-RNN Tuning-Netherland-2', 'RFF-GRU Tuning-Netherland-2', 'RFF-LSTM Tuning-Netherland-2']
  ✅ Contour + grouped importances saved in Netherland-2/

📊 Processing: Synthetic


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


['RFF-RNN Tuning-Synthetic', 'RFF-GRU Tuning-Synthetic', 'RFF-LSTM Tuning-Synthetic']
  ✅ Contour + grouped importances saved in Synthetic/


In [5]:
results_data = []

for db_file in files:
    storage_name = f"sqlite:///{db_file.absolute()}"
    available_studies = optuna.study.get_all_study_names(storage=storage_name)
    
    dataset_name = db_file.stem.replace('optuna_results_', '')
    
    for study_name in available_studies:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        
        # Extraer tipo de modelo del nombre del estudio
        if 'RNN' in study_name and 'LSTM' not in study_name and 'GRU' not in study_name:
            model_type = 'RNN'
        elif 'GRU' in study_name:
            model_type = 'GRU'
        elif 'LSTM' in study_name:
            model_type = 'LSTM'
        else:
            model_type = 'Unknown'
        
        # Obtener el mejor trial
        best_trial = study.best_trial
        
        # Crear registro con dataset, modelo, métrica y parámetros
        record = {
            'Dataset': dataset_name,
            'Model': model_type,
            'Best_RMSE': best_trial.value,  # Asumiendo que optimizaste RMSE
            **best_trial.params  # Desempacar todos los hiperparámetros
        }
        
        results_data.append(record)

# Crear DataFrame
df_hyperparameters = pd.DataFrame(results_data)

# Ordenar por Dataset y Modelo
df_hyperparameters = df_hyperparameters.sort_values(['Dataset', 'Model']).reset_index(drop=True)

# Mostrar la tabla
print("\n" + "="*100)
print("TABLA DE HIPERPARÁMETROS ÓPTIMOS POR MODELO Y DATASET")
print("="*100)
display(df_hyperparameters)

# Opcional: Guardar como CSV
csv_path = PLOTS_DIR / "optimal_hyperparameters.csv"
df_hyperparameters.to_csv(csv_path, index=False)
print(f"\n✅ Tabla guardada en: {csv_path}")


TABLA DE HIPERPARÁMETROS ÓPTIMOS POR MODELO Y DATASET


,Dataset,Model,Best_RMSE,lr_rho_multiplier,bands_preset,nf_per_band,hidden,num_layers,spectral_dropout_p
0,Argone,GRU,0.003921,0.954783,12-24-168,24,256,3,0.1
1,Argone,LSTM,0.006633,3.856406,12-24-168,104,144,3,0.2
2,Argone,RNN,0.007021,0.580793,6-24-72,48,32,3,0.2
3,Beijing,GRU,0.004110,0.954783,12-24-168,24,256,3,0.1
4,Beijing,LSTM,0.006915,3.856406,12-24-168,104,144,3,0.2
5,Beijing,RNN,0.009173,0.580793,6-24-72,48,32,3,0.2
6,Chengdu,GRU,0.003547,0.954783,12-24-168,24,256,3,0.1
7,Chengdu,LSTM,0.004483,3.648156,4-24-168,48,96,3,0.3
8,Chengdu,RNN,0.006181,1.184432,4-24-168,32,48,1,0.5
9,Netherland-0,GRU,0.007266,0.613003,12-24-168,56,80,3,0.2



✅ Tabla guardada en: ../results/plots/optimal_hyperparameters.csv
